In [1]:
import os
import numpy as np #for general math
import scipy as sp #for statistical analysis and optimization
import nibabel as nib #for handling the given MRI format
from nibabel.testing import data_path
import imageio #for saving images to files
import matplotlib.pyplot as plt #for displaying images
from PIL import Image #for image processing and transformation functions
from scipy import optimize
import pandas as pd #for storing data
import skimage as ski #for image processing and transformation functions
import skimage.transform as tf
from skimage import filters
import cv2 #I was playing cv2's functions and filters
from skimage.filters.rank import mean_bilateral #for pre-processing
from skimage.filters.rank import enhance_contrast #for pre-processing
from skimage.filters.rank import enhance_contrast_percentile #for pre-processing
from skimage.morphology import disk #for pre-processing
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral, #for pre-processing
                                 denoise_wavelet, estimate_sigma)
import SimpleITK as sitk #non-linear diffeomorphic image registration

#changing matplotlib image display format
%matplotlib inline 
%config InlineBackend.figure_format = 'svg'

params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

In [2]:
#load the data from milestone 1
MS1Data = np.load('MS1Data.npy', allow_pickle=True)

'''
    - The images are min maxed normalized between 0 and 255
    - img->MRI scan
    - seg->Segmentation
'''

#Training Data

S1img = np.ndarray.astype(np.array([MS1Data[0][0]/np.max(MS1Data[0][0])*255]), np.int)
S1seg = MS1Data[1]
S2img = np.ndarray.astype(np.array([MS1Data[2][0]/np.max(MS1Data[2][0])*255]), np.int)
S2seg = MS1Data[3]
S3img = np.ndarray.astype(np.array([MS1Data[4][0]/np.max(MS1Data[4][0])*255]), np.int)
S3seg = MS1Data[5]
S4img = np.ndarray.astype(np.array([MS1Data[6][0]/np.max(MS1Data[6][0])*255]), np.int)
S4seg = MS1Data[7]
S5img = np.ndarray.astype(np.array([MS1Data[8][0]/np.max(MS1Data[8][0])*255]), np.int)
S5seg = MS1Data[9]
S6img = np.ndarray.astype(np.array([MS1Data[10][0]/np.max(MS1Data[10][0])*255]), np.int)
S6seg = MS1Data[11]

#Validation Data

S7img = np.ndarray.astype(np.array([MS1Data[12][0]/np.max(MS1Data[12][0])*255]), np.int)
S7seg = MS1Data[13]
S15img = np.ndarray.astype(np.array([MS1Data[14][0]/np.max(MS1Data[14][0])*255]), np.int)
S15seg = MS1Data[15]

# Load the volumetric Data Because we might use that for registration also

VolsT = np.load('VolumesTraining.npy', allow_pickle=True)

V1img = np.ndarray.astype(np.array([VolsT[0]/np.max(VolsT[0])*255]), np.int)[0]
V1seg = VolsT[1]
V2img = np.ndarray.astype(np.array([VolsT[2]/np.max(VolsT[2])*255]), np.int)[0]
V2seg = VolsT[3]
V3img = np.ndarray.astype(np.array([VolsT[4]/np.max(VolsT[4])*255]), np.int)[0]
V3seg = VolsT[5]
V4img = np.ndarray.astype(np.array([VolsT[6]/np.max(VolsT[6])*255]), np.int)[0]
V4seg = VolsT[7]
V5img = np.ndarray.astype(np.array([VolsT[8]/np.max(VolsT[8])*255]), np.int)[0]
V5seg = VolsT[9]
V6img = np.ndarray.astype(np.array([VolsT[10]/np.max(VolsT[10])*255]), np.int)[0]
V6seg = VolsT[11]
V7img = np.ndarray.astype(np.array([VolsT[12]/np.max(VolsT[12])*255]), np.int)[0]
V7seg = VolsT[13]
V15img = np.ndarray.astype(np.array([VolsT[14]/np.max(VolsT[14])*255]), np.int)[0]
V15seg = VolsT[15]

In [78]:
def GlobalAlign3D(fixed, moving, interp = sitk.sitkLinear):
    R = sitk.ImageRegistrationMethod()
    
    R.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=500, 
                                                      convergenceMinimumValue=1e-5, 
                                                      convergenceWindowSize=10)
    
    R.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    R.SetMetricSamplingStrategy(R.RANDOM)
    R.SetMetricSamplingPercentage(0.01)
    R.SetInterpolator(sitk.sitkLinear)

    R.SetOptimizerScalesFromPhysicalShift() 
    R.SetInitialTransform(sitk.Similarity3DTransform(), inPlace=False)
    R.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    R.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
    R.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    
    
    outTx = R.Execute(fixed, moving)
    
    out = sitk.Resample(moving, outTx, interp)
    
    return (outTx, out)

In [71]:
'''
This is function that performs diffeomorphic image registration using the DEMONs algorithm. This is monte carlo algrothm
which works by making random changes by simulating force vectors on the coordinate grid for the moving image 
with guided random pertubations. This is repeated until it converges to the fixed image
To learn more, URL: https://simpleitk.readthedocs.io/en/master/link_DemonsRegistration1_docs.html

    params:
        fixed - image we want to transform the moving image into
        moving - image we are transforming into fixed
        interp - interpolation method we want to use for transformations
            Nearest Neighbor: sitk.sitkNearestNeighbor
            Linear: sitk.sitkLinear
            BSpline: sitk.sitkBSpline
            Gaussian: sitk.sitkGaussian
            Sinc: sitk.sitkHammingWindowedSinc
            etc. (look on the SITK documentation website for more)
    returns:
        (diffeomorphic transform parameters, transformed image)

'''
def diffeomorphicRegistration(fixed, moving, interp = sitk.sitkNearestNeighbor):
    
    # Perform the Demons diffeomorphic registration
    demons = sitk.DiffeomorphicDemonsRegistrationFilter()
    
    # Number of Iterations is 100
    demons.SetNumberOfIterations( 500 )
    
    # Standard deviation for Gaussian smoothing of displacement field
        # Determines how much deformation we allow in our diffeomorphism
    demons.SetStandardDeviations( 2.0 )
    
    # Guarantees a smooth displacement field
    demons.SmoothDisplacementFieldOn()
    
    displacementField = demons.Execute( fixed, moving ) #execute the registration
    
    # Output transformations and set the interpolator to
    outTx = sitk.DisplacementFieldTransform( displacementField )
    outTx.SetInterpolator(interp)
    
    # Find the moving image under this transformation
    
    out = sitk.Resample(moving, outTx, interp)
    
    return (outTx, out)

In [72]:
'''
This function takes in an image and smooths it using a gaussian kernal with a provided sigma.
After smoothing, the image is shrunk by a given shrink factor. Note that SimpleITK actualy superimposes
physical coordinates and space on an image, but this strinking refers to acutal image data size rather 
than spatial size. Spatial size is kept constant in this implementation below.

    params:
        image - image we want to smooth and shrink
        shrink_factor - the amount we want to shrink the image by in image data, -> newsize = oldsize/shrink_factor
            Note: Spatial Dimensions are kept the same
        smoothing_sigma - standard deviation of gaussian smoothing kernal
    returns:
        Smoothed and shrunk image

'''
def smooth_and_resample(image, shrink_factor, smoothing_sigma):
    # Smooth the image using a gaussian kernal with the standard deviation
        #"smoothing_sigma"
    smoothed_image = sitk.SmoothingRecursiveGaussian(image, smoothing_sigma)
    
    # Get the shape and the spacing for the image we want to shrink
        # Both of these values are tuples in the dimension of the image (size 2)
    spacing = image.GetSpacing()
    size = image.GetSize()
    
    # Scale the image down by the shrink factor (with a ceiling operator) in each of the dimensions
    new_size = [int(size[0]/float(shrink_factor) + 0.5), int(size[1]/float(shrink_factor) + 0.5)]
    # Here are the new spacings for the image, get absolute size of image by multiplying
        # size by the spacing and dividing by the new shrunk size
    new_spacing = [(size[0]-1)*spacing[0]/(new_size[0]-1), (size[0]-1)*spacing[0]/(new_size[0]-1)]
    
    outimg = sitk.Resample(smoothed_image, new_size, sitk.Transform(), 
                         sitk.sitkLinear, image.GetOrigin(), # use the original image origin and direction
                         new_spacing, image.GetDirection(), 0.0, # Default pixel value is 0.0
                         image.GetPixelID())
    return outimg

In [79]:
'''
This is a multiscale diffeomorphic transformation that uses an image pyramid to find a better
diffeomorphic transformation. The idea of an image pyramid is to decompose an image into multiple scales
and then extract data at each given scale. The reason we want to do this is because if there is noise
in the image, this multpile scale extraction will help reduce the effects of noise on the image. First we
perform the registration on the smallest scale image and then we use the resultant diplacement field
as the original displacement field for the next scale and work our way down to the full resolution. This should
prevent noise from affecting our final transformation. This pyramid uses gaussian filtering at each step.
Here is the reference material for this code:
URL: https://simpleitk.org/SPIE2019_COURSE/05_advanced_registration.html
    params:
        fixed - image we want to transform the moving image into
        moving - image we are transforming into fixed
        interp - interpolation method we want to use for transformations
            Nearest Neighbor: sitk.sitkNearestNeighbor
            Linear: sitk.sitkLinear
            BSpline: sitk.sitkBSpline
            Gaussian: sitk.sitkGaussian
            Sinc: sitk.sitkHammingWindowedSinc
            etc. (look on the SITK documentation website for more)
    returns:
        (diffeomorphic transform parameters, transformed image)

'''
def multiscale_diffeomorphic(fixed_image, moving_image, interp = sitk.sitkLinear):
    # Initialize the shrink factors and smoothing sigma for our image pyramid
    shrink_factors = [4, 2, 1]
    smoothing_sigmas = [2, 1, 0]
    
    # Initialize the image pyramid
    fixed_images = [fixed_image]
    moving_images = [moving_image]
    
    # Add all images smoothed to the levels and shrunk to the levels above
        # Note that we flip the order in which the pyramid is structured, so the smallest image is first
    for shrink_factor, smoothing_sigma in reversed(list(zip(shrink_factors, smoothing_sigmas))):
            fixed_images.append(smooth_and_resample(fixed_images[0], shrink_factor, smoothing_sigma))
            moving_images.append(smooth_and_resample(moving_images[0], shrink_factor, smoothing_sigma))
    
    # Make our initial displacement field for the diffeomorphic transform
        # Based on the last element in the array so should be the one with shrink 8, smooth 8
    initial_displacement_field = sitk.Image(fixed_images[-1].GetWidth(), 
                                                fixed_images[-1].GetHeight(),
                                                fixed_images[-1].GetDepth(),
                                                sitk.sitkVectorFloat64)
    initial_displacement_field.CopyInformation(fixed_images[-1]) # Use the information from the fixed image
    
    # Set out type of transformation to diffeomorphic registration
    demons = sitk.DiffeomorphicDemonsRegistrationFilter()
    demons.SetNumberOfIterations(200)
    demons.SmoothDisplacementFieldOn()
    demons.SetStandardDeviations(1.0)
    demons.SmoothUpdateFieldOff()
    
    # Execute the registration
    initial_displacement_field = demons.Execute(fixed_images[-1], moving_images[-1], initial_displacement_field)
    
    # Start at the top of the pyramid and work our way down.    
    for f_image, m_image in reversed(list(zip(fixed_images[0:-1], moving_images[0:-1]))):
            initial_displacement_field = sitk.Resample (initial_displacement_field, f_image)
            initial_displacement_field = demons.Execute(f_image, m_image, initial_displacement_field)
            
    # We have our final transformation
    outTx = sitk.DisplacementFieldTransform(initial_displacement_field)
    
    # Perform the transformation on the original moving image
    out = sitk.Resample(moving_image, outTx, interp)
    
    return (outTx, out)

In [74]:
'''
Computes pearson normalized correlation between timg and img at pixel at index = (i,j) in a 11x11 neighborhood
        params:
            index - pixel index in array we are performing correlation on
            timg, img - training and target image we perform correlation on
        returns:
            perason correlation of two images at the pixel at index in a 11x11 grid
'''
def corr(index, timg, img):
    # Zero padding for 11x11 grid
    # Renormalize coordinate/index to this new grid
    img = np.pad(img, ((5,5),(5,5)), 'constant')
    timg = np.pad(timg, ((5,5),(5,5)), 'constant')
    coord = np.array(index) + 5
    i = coord[0]
    j = coord[1]
    
    #only consider pixels that are nonzero on both images timg and img and apply the 11x11 grid
    mask = (img != 0) & (timg != 0)
    img = (img*mask)[i-5:i+6, j-5:j+6]
    timg = (timg*mask)[i-5:i+6, j-5:j+6]
    
    # Return the correlation
    result = sp.stats.pearsonr(np.ndarray.flatten(img), np.ndarray.flatten(timg))[0]
    
    # If the correlation is NAN, just make the weight 1 (this is the case for constant / zero chunks)
    if(np.isnan(result)):
        result = 1
    return result


In [75]:
'''
Correlation weighted voting Segmentation
        params:
            td - stack of all candidate segmentations
            img - image we want to perform segmentation on
            timg - stack of all transformed traning data
        returns:
            A final segmentation result
'''
def corSeg(td, img, timg):
    # First initialize an empty image in the desired shape of our segmentation
    shape = td[0].shape
    result = np.zeros(shape)
    
    # For each pixel in the resultant segmentation, determine the true value of the pixel intensity
    for i in range(shape[0]):
        for j in range(shape[1]): 
            #For every pixel in the image find most frequent pixel segmentation weighted by correlation
            arr = np.zeros(6, dtype = np.int)
            weights = []
            for k in range(0,6):
                arr[k] = (td[k])[i][j]
                weights.append(corr([i,j], timg[k], img))
            # Negative weights don't make sense so add 1 such that the smallest possible weight is 0
                # View this as shifting over your distribution by 1
            weights = np.array(weights) + 1
            bins = np.bincount(np.array(arr), weights = weights)
            
            result[i][j] = np.argmax(bins)
            
    return result

In [76]:
#Here is a dictionary of all of the ROIs that we care about
ROIs = {'Left-Cerebral-Exterior': 1,
            'Left-Cerebral-White-Matter': 2,
            'Left-Cerebral-Cortex': 3,
            'Left-Lateral-Ventricle': 4,
            'Left-Inf-Lat-Vent': 5,
            'Left-Cerebellum-Exterior': 6,
            'Left-Cerebellum-White-Matter': 7,
            'Left-Cerebellum-Cortex': 8,
            'Left-Thalamus': 9,
            'Left-Thalamus-Proper': 10,
            'Left-Caudate': 11,
            'Left-Putamen': 12,
            'Left-Pallidum': 13,
            '3rd-Ventricle': 14,
            '4th-Ventricle': 15,
            'Brain-Stem': 16,
            'Left-Hippocampus': 17,
            'Left-Amygdala': 18,
            'Left-Insula': 19,
            'Left-Operculum': 20,
            'Line-1': 21,
            'Line-2': 22,
            'Line-3': 23,
            'CSF': 24,
            'Left-Lesion': 25,
            'Left-Accumbens-area': 26,
            'Left-Substancia-Nigra': 27,
            'Left-VentralDC': 28,
            'Left-undetermined': 29,
            'Left-vessel': 30,
            'Left-choroid-plexus': 31,
            'Left-F3orb': 32,
            'Left-lOg': 33,
            'Left-aOg': 34,
            'Left-mOg': 35,
            'Left-pOg': 36,
            'Left-Stellate': 37,
            'Left-Porg': 38,
            'Left-Aorg': 39,
            'Right-Cerebral-Exterior': 40,
            'Right-Cerebral-White-Matter': 41,
            'Right-Cerebral-Cortex': 42,
            'Right-Lateral-Ventricle': 43,
            'Right-Inf-Lat-Vent': 44,
            'Right-Cerebellum-Exterior': 45,
            'Right-Cerebellum-White-Matter': 46,
            'Right-Cerebellum-Cortex': 47,
            'Right-Thalamus': 48,
            'Right-Thalamus-Proper': 49,
            'Right-Caudate': 50,
            'Right-Putamen': 51,
            'Right-Pallidum': 52,
            'Right-Hippocampus': 53,
            'Right-Amygdala': 54,
            'Right-Insula': 55,
            'Right-Operculum': 56,
            'Right-Lesion': 57,
            'Right-Accumbens-area': 58,
            'Right-Substancia-Nigra': 59,
            'Right-VentralDC': 60
           }

In [77]:
'''
Calculates the Jacaard Overlap (intersection over union) an automatic and manual segmentation for a specific ROI
    params:
        auto - automatic segmentation image
        manual - manual segmentation image
        r - region of interest based on the dictionary in the above cell
    returns:
        Jaacard overlap for the two segmentations for ROI = r
'''
def JacOverlap(auto, manual, r):
    # Uses bitwise operators and '==' operators to make this function run faster
    r = ROIs[r]
    # All the pixels in auto and manual that are in r
    auto = auto == r
    manual = manual == r
    
    # Intersection is the bitwise 'and' of the two matrices above
    inter = np.sum(auto & manual)
    
    # Union is the bitwise 'or' of the two matrices above
    union = np.sum(auto | manual)
    
    return inter/union

In [81]:
def PFinal(img, imgSpacing, manualSeg = None):
    imgV = sitk.GetImageFromArray(img/np.max(img), isVector = False)
    imgV.SetSpacing(imgSpacing) 
    V1 = sitk.GetImageFromArray(V1img/np.max(V1img), isVector = False)
    V1.SetSpacing([0.9375, 0.9375, 1.5])
    V1 = sitk.Resample(V1, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    V2 = sitk.GetImageFromArray(V2img/np.max(V2img), isVector = False)
    V2.SetSpacing([0.9375, 0.9375, 1.5])
    V2 = sitk.Resample(V2, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    V3 = sitk.GetImageFromArray(V3img/np.max(V3img), isVector = False)
    V3.SetSpacing([0.9375, 0.9375, 1.5])
    V3 = sitk.Resample(V3, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    V4 = sitk.GetImageFromArray(V4img/np.max(V4img), isVector = False)
    V4.SetSpacing([0.9375, 0.9375, 1.5])
    V4 = sitk.Resample(V4, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    V5 = sitk.GetImageFromArray(V5img/np.max(V5img), isVector = False)
    V5.SetSpacing([0.9375, 0.9375, 1.5])
    V5 = sitk.Resample(V5, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    V6 = sitk.GetImageFromArray(V6img/np.max(V6img), isVector = False)
    V6.SetSpacing([0.9375, 0.9375, 1.5])
    V6 = sitk.Resample(V6, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkLinear, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max1 = np.max(V1seg)
    S1 = sitk.GetImageFromArray(V1seg/np.max(V1seg), isVector = False)
    S1.SetSpacing([0.9375, 0.9375, 1.5])
    S1 = sitk.Resample(S1, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max2 = np.max(V2seg)
    S2 = sitk.GetImageFromArray(V2seg/np.max(V2seg), isVector = False)
    S2.SetSpacing([0.9375, 0.9375, 1.5])
    S2 = sitk.Resample(S2, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max3 = np.max(V3seg)
    S3 = sitk.GetImageFromArray(V3seg/np.max(V3seg), isVector = False)
    S3.SetSpacing([0.9375, 0.9375, 1.5])
    S3 = sitk.Resample(S3, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max4 = np.max(V4seg)
    S4 = sitk.GetImageFromArray(V4seg/np.max(V4seg), isVector = False)
    S4.SetSpacing([0.9375, 0.9375, 1.5])
    S4 = sitk.Resample(S4, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max5 = np.max(V5seg)
    S5 = sitk.GetImageFromArray(V5seg/np.max(V5seg), isVector = False)
    S5.SetSpacing([0.9375, 0.9375, 1.5])
    S5 = sitk.Resample(S5, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    max6 = np.max(V6seg)
    S6 = sitk.GetImageFromArray(V6seg/np.max(V6seg), isVector = False)
    S6.SetSpacing([0.9375, 0.9375, 1.5])
    S6 = sitk.Resample(S6, imgV.GetSize(), sitk.Transform(), 
                         sitk.sitkNearestNeighbor, imgV.GetOrigin(),
                         imgV.GetSpacing(), imgV.GetDirection(), 0.0,
                         imgV.GetPixelID())
    
    
    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(1024) # We use 1024 bins
    matcher.SetNumberOfMatchPoints(7) # 7 quantiles for matching
    matcher.ThresholdAtMeanIntensityOn() # Use mean thresholding to get rid of background
    
    V1 = matcher.Execute(V1, imgV)
    V2 = matcher.Execute(V2, imgV)
    V3 = matcher.Execute(V3, imgV)
    V4 = matcher.Execute(V4, imgV)
    V5 = matcher.Execute(V5, imgV)
    V6 = matcher.Execute(V6, imgV)
    
    aff1 = GlobalAlign3D(imgV, V1)
    aff2 = GlobalAlign3D(imgV, V2)
    aff3 = GlobalAlign3D(imgV, V3)
    aff4 = GlobalAlign3D(imgV, V4)
    aff5 = GlobalAlign3D(imgV, V5)
    aff6 = GlobalAlign3D(imgV, V6)
    
    S1 = sitk.Resample(S1, aff1[0], sitk.sitkNearestNeighbor)
    S2 = sitk.Resample(S2, aff2[0], sitk.sitkNearestNeighbor)
    S3 = sitk.Resample(S3, aff3[0], sitk.sitkNearestNeighbor)
    S4 = sitk.Resample(S4, aff4[0], sitk.sitkNearestNeighbor)
    S5 = sitk.Resample(S5, aff5[0], sitk.sitkNearestNeighbor)
    S6 = sitk.Resample(S6, aff6[0], sitk.sitkNearestNeighbor)
    
    diff1 = diffeomorphicRegistration(imgV, aff1[1])
    diff2 = diffeomorphicRegistration(imgV, aff2[1])
    diff3 = diffeomorphicRegistration(imgV, aff3[1])
    diff4 = diffeomorphicRegistration(imgV, aff4[1])
    diff5 = diffeomorphicRegistration(imgV, aff5[1])
    diff6 = diffeomorphicRegistration(imgV, aff6[1])
    
    S1 = sitk.Resample(S1, diff1[0], sitk.sitkNearestNeighbor)
    S2 = sitk.Resample(S2, diff2[0], sitk.sitkNearestNeighbor)
    S3 = sitk.Resample(S3, diff3[0], sitk.sitkNearestNeighbor)
    S4 = sitk.Resample(S4, diff4[0], sitk.sitkNearestNeighbor)
    S5 = sitk.Resample(S5, diff5[0], sitk.sitkNearestNeighbor)
    S6 = sitk.Resample(S6, diff6[0], sitk.sitkNearestNeighbor)
    
    diff1 = sitk.GetArrayFromImage(diff1[1])
    diff2 = sitk.GetArrayFromImage(diff2[1])
    diff3 = sitk.GetArrayFromImage(diff3[1])
    diff4 = sitk.GetArrayFromImage(diff4[1])
    diff5 = sitk.GetArrayFromImage(diff5[1])
    diff6 = sitk.GetArrayFromImage(diff6[1])
    
    S1 = sitk.GetArrayFromImage(S1)
    S2 = sitk.GetArrayFromImage(S2)
    S3 = sitk.GetArrayFromImage(S3)
    S4 = sitk.GetArrayFromImage(S4)
    S5 = sitk.GetArrayFromImage(S5)
    S6 = sitk.GetArrayFromImage(S6)
    
    diff1 = diff1[:, :, int(diff1.shape[2]/2 - 1)]
    plt.imshow(diff1)
    plt.show()
    
    d1 = sitk.GetImageFromArray(diff1, sitk.sitkFloat32)
    d1 = sitk.VectorIndexSelectionCast(d1,0)
    
    diff2 = diff2[:, :, int(diff2.shape[2]/2 - 1)]
    plt.imshow(diff2)
    plt.show()
    
    d2 = sitk.GetImageFromArray(diff2, sitk.sitkFloat32)
    d2 = sitk.VectorIndexSelectionCast(d2,0)
    
    diff3 = diff3[:, :, int(diff3.shape[2]/2 - 1)]
    plt.imshow(diff3)
    plt.show()
    
    d3 = sitk.GetImageFromArray(diff3, sitk.sitkFloat32)
    d3 = sitk.VectorIndexSelectionCast(d3,0)
    
    diff4 = diff4[:, :, int(diff4.shape[2]/2 - 1)]
    plt.imshow(diff4)
    plt.show()
    
    d4 = sitk.GetImageFromArray(diff4, sitk.sitkFloat32)
    d4 = sitk.VectorIndexSelectionCast(d4,0)
    
    diff5 = diff5[:, :, int(diff5.shape[2]/2 - 1)]
    plt.imshow(diff5)
    plt.show()
    
    d5 = sitk.GetImageFromArray(diff5, sitk.sitkFloat32)
    d5 = sitk.VectorIndexSelectionCast(d5,0)
    
    diff6 = diff6[:, :, int(diff6.shape[2]/2 - 1)]
    plt.imshow(diff6)
    plt.show()
    
    d6 = sitk.GetImageFromArray(diff6, sitk.sitkFloat32)
    d6 = sitk.VectorIndexSelectionCast(d6,0)
    
    
    S1 = S1[:, :, int(S1.shape[2]/2 - 1)]
    plt.imshow(S1)
    plt.show()
    
    l1 = sitk.GetImageFromArray(S1, sitk.sitkFloat32)
    l1 = sitk.VectorIndexSelectionCast(l1,0)
    
    S2 = S2[:, :, int(S2.shape[2]/2 - 1)]
    plt.imshow(S2)
    plt.show()
    
    l2 = sitk.GetImageFromArray(S2, sitk.sitkFloat32)
    l2 = sitk.VectorIndexSelectionCast(l2,0)
    
    S3 = S3[:, :, int(S3.shape[2]/2 - 1)]
    plt.imshow(S3)
    plt.show()
    
    l3 = sitk.GetImageFromArray(S3, sitk.sitkFloat32)
    l3 = sitk.VectorIndexSelectionCast(l3,0)
    
    S4 = S4[:, :, int(S4.shape[2]/2 - 1)]
    plt.imshow(S4)
    plt.show()
    
    l4 = sitk.GetImageFromArray(S4, sitk.sitkFloat32)
    l4 = sitk.VectorIndexSelectionCast(l4,0)
    
    S5 = S5[:, :, int(S5.shape[2]/2 - 1)]
    plt.imshow(S5)
    plt.show()
    
    l5 = sitk.GetImageFromArray(S5, sitk.sitkFloat32)
    l5 = sitk.VectorIndexSelectionCast(l5,0)
    
    S6 = S6[:, :, int(S6.shape[2]/2 - 1)]
    plt.imshow(S6)
    plt.show()
    
    l6 = sitk.GetImageFromArray(S6, sitk.sitkFloat32)
    l6 = sitk.VectorIndexSelectionCast(l6,0)
    
    img = np.ndarray.astype(img[:, :, int(img.shape[2]/2 - 1)], np.float)
    imgV = sitk.GetImageFromArray(img, sitk.sitkFloat32)
    imgV = sitk.VectorIndexSelectionCast(imgV,0)
    
    ms1 = multiscale_diffeomorphic(imgV, d1)
    ms2 = multiscale_diffeomorphic(imgV, d2)
    ms3 = multiscale_diffeomorphic(imgV, d3)
    ms4 = multiscale_diffeomorphic(imgV, d4)
    ms5 = multiscale_diffeomorphic(imgV, d5)
    ms6 = multiscale_diffeomorphic(imgV, d6)
    
    S1 = sitk.Resample(l1, ms1[0], sitk.sitkNearestNeighbor)
    S2 = sitk.Resample(l2, ms2[0], sitk.sitkNearestNeighbor)
    S3 = sitk.Resample(l3, ms3[0], sitk.sitkNearestNeighbor)
    S4 = sitk.Resample(l4, ms4[0], sitk.sitkNearestNeighbor)
    S5 = sitk.Resample(l5, ms5[0], sitk.sitkNearestNeighbor)
    S6 = sitk.Resample(l6, ms6[0], sitk.sitkNearestNeighbor)
    
    S = np.ndarray.astype(np.array([sitk.GetArrayFromImage(S1)*max1, sitk.GetArrayFromImage(S2)*max2,
                 sitk.GetArrayFromImage(S3)*max3, sitk.GetArrayFromImage(S4)*max4,
                 sitk.GetArrayFromImage(S5)*max5, sitk.GetArrayFromImage(S6)*max6]), np.int)
    
    R = np.array([sitk.GetArrayFromImage(ms1[1]), sitk.GetArrayFromImage(ms2[1]),
                 sitk.GetArrayFromImage(ms3[1]), sitk.GetArrayFromImage(ms4[1]),
                 sitk.GetArrayFromImage(ms5[1]), sitk.GetArrayFromImage(ms6[1])])
    
    result = corSeg(td = S, timg = R, img = img)
    
    if(manualSeg is None):
        return result
    
    # Else calculate Jaccard Overlap
    JacLCWM = JacOverlap(result, manualSeg, 'Left-Cerebral-White-Matter')
    JacRCWM = JacOverlap(result, manualSeg, 'Right-Cerebral-White-Matter')
    JacLCC = JacOverlap(result, manualSeg, 'Left-Cerebral-Cortex')
    JacRCC = JacOverlap(result, manualSeg, 'Right-Cerebral-Cortex')
    
    Jacs = [[JacLCWM, JacRCWM], [JacLCC, JacRCC]]
    Jacsdf = pd.DataFrame(Jacs, columns = ['Left', 'Right'], index = ['C. White Matter', 'C. Cortex'])
    return (result, Jacsdf)
    